In [3]:
# !pip install cvxopt
import cvxopt
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.datasets import load_digits
from sklearn.utils import shuffle
import random

In [4]:
# Section A

all_X, all_y = load_digits(return_X_y=True)

all_X = np.array(all_X)
all_y = np.array(all_y)
all_X, all_y = shuffle(all_X, all_y)

def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0: 
        return v
    return v / norm

all_X = np.apply_along_axis(normalize, axis=1, arr=all_X)


len_test = int(0.1*len(all_X))
len_train = len(all_X) - len_test
test_ind = np.array([np.random.choice(np.where(all_y == i)[0], 15, replace=False) for i in range(10)]).reshape(1, 150)
test_ind = np.concatenate((test_ind, 
                           np.random.choice(np.setdiff1d(range(len(all_y)), 
                                                         test_ind), len_test - 150, replace=False)), axis=None)
test_X, test_Y = all_X[test_ind], all_y[test_ind]
train_X, train_Y = all_X[np.setdiff1d(range(len(all_y)), test_ind)], all_y[np.setdiff1d(range(len(all_y)), test_ind)]



In [5]:
class LinearKernel:
    def __init__(self, theta=None):
        pass
    
    def __call__(self, X, Y):
        return X @ Y.T
    
    
class GaussianKernel:
    def __init__(self, theta):
        self.theta = theta
        
    def __call__(self, X, Y):
        if (X.ndim == 1) and (Y.ndim == 1):
            tmp = np.linalg.norm(X - Y)**2
        elif ((X.ndim == 1) and (Y.ndim != 1)) or ((X.ndim != 1) and (Y.ndim == 1)):
            tmp = np.linalg.norm(X - Y, axis=1)**2
        else:
            tmp = np.reshape(np.sum(X**2,axis=1), (len(X), 1)) + np.sum(Y**2, axis=1)  -2 * (X @ Y.T)
        K = np.exp(- tmp/(2*self.theta**2))
        return K

In [6]:
def cvxopt_solve_qp(P, q, G=None, h=None, A=None, b=None):
    
#     P = .5 * (P + P.T)  # make sure P is symmetric
    args = [cvxopt.matrix(P), cvxopt.matrix(q)]
    if G is not None:
        args.extend([cvxopt.matrix(G), cvxopt.matrix(h)])
        if A is not None:
            args.extend([cvxopt.matrix(A), cvxopt.matrix(b)])
            

    #Setting solver parameters (change default to decrease tolerance) 
    cvxopt.solvers.options['show_progress'] = False
    cvxopt.solvers.options['abstol'] = 1e-10
    cvxopt.solvers.options['reltol'] = 1e-10
    cvxopt.solvers.options['feastol'] = 1e-10

    sol = cvxopt.solvers.qp(*args)
    if 'optimal' not in sol['status']:
        return None
    
    return np.array(sol['x']).reshape(1, -1)[0]


class SVC:
    def __init__(self, kernel, C=1.0):
        self.C = C
        self.kernel = kernel
          
    def fit(self, X, t, tol_conv=1e-6, tol_sv=1e-4, show_time = False): 

        n,m = X.shape
        y = t.reshape(-1,1) * 1.
        X_dash = y * X
        H = np.dot(X_dash , X_dash.T) * 1.
        H = self.kernel(X, X) * self.kernel(y, y)
        
        # Converting into cvxopt format 
        P = cvxopt.matrix(H)
        q = cvxopt.matrix(-np.ones((n, 1)))
        G = cvxopt.matrix(np.vstack((np.eye(n)*-1, np.eye(n))))
        h = cvxopt.matrix(np.hstack((np.zeros(n), np.ones(n) * self.C)))
        A = cvxopt.matrix(y.reshape(1, -1))
        b = cvxopt.matrix(np.zeros(1))
        
        a = cvxopt_solve_qp(P=P, q=q, G=G, h=h, A=A, b=b)
        ind_sv = np.where(a > tol_sv)
        self.a = a[ind_sv]
        self.X_sv = X[ind_sv]
        self.t_sv = t[ind_sv]
        b = self.t_sv - self.kernel(self.X_sv, self.X_sv) @ (self.a*self.t_sv)
        self.b = sum(b) / len(b)
        
        ind_unbounded_sv = np.where( np.round(self.a, 2) < self.C)
        self.ind_unbounded_sv = ind_unbounded_sv
        unbounded_X_sv = self.X_sv[ind_unbounded_sv]
        unbounded_t_sv = self.t_sv[ind_unbounded_sv]
        b = unbounded_t_sv - self.kernel(unbounded_X_sv, self.X_sv) @ (self.a*self.t_sv)
        if len(b):
            self.b = sum(b) / len(b)
        
        self.coef_ = self.X_sv.T @ (self.a*self.t_sv)
        self.support_ = ind_sv
        self.n_support_ = [np.count_nonzero(self.t_sv < 0), np.count_nonzero(self.t_sv > 0)]
            
    def decision_function(self, X):
        val_dec_func = self.kernel(X, self.X_sv) @ (self.a*self.t_sv) + self.b
        return val_dec_func 
    
    def predict(self,X):
        pred = np.sign(self.decision_function(X))
        return pred
    
    def details(self):
        print("____________")
        if type(self.kernel) == LinearKernel:
            print('w = ', self.coef_)
        print('b = ', self.b)
        print('Indices of support vectors = ', self.support_)
        # print('Support vectors = ', clf.support_vectors_)
        print('Number of support vectors for each class = ', self.n_support_)
        print('Coefficients of the support vector in the decision function = ', self.a)
        print()
        

class MultiClass_SVC:
    def __init__(self, kernel, C=1.0, num_lables=10):
        self.C = C
        self.kernel = kernel
        self.num_lables = num_lables
    
    def fit(self, X, y, tol_conv=1e-6, tol_sv=1e-4, show_time = False): 
        self.train_X = X
        self.train_y = y
        self.classifiers = [None for i in range(self.num_lables)]
        for num in range(0, self.num_lables):
            train_y = np.where(y == num, 1, -1).astype(int)
            svc = SVC(C=self.C, kernel=self.kernel)
            svc.fit(X, train_y)
            self.classifiers[num] = svc
            
    def decision_function(self, X):
        classProbabilities = np.array([self.classifiers[i].decision_function(X) 
                                       for i in range(self.num_lables)]).T
        return classProbabilities 
    
    def predict(self,X):
        classProbabilities = self.decision_function(X)
        pred = classProbabilities.argmax(axis=1)
        return pred
    
    def get_accuracy(self, X, Y):
        return np.sum(Y == self.predict(X)) / len(Y)
    
    def print_accuracy(self, train_X, train_y, test_X, test_y):
        print(f"train accuracy score: {self.get_accuracy(train_X, train_y)}")
        print(f"test accuracy score: {self.get_accuracy(test_X, test_y)}")
    
    def print_confusion_matrix(self, _X, _y):    
        predicts = self.predict(_X)
        confusion_matrix = [[0 for i in range(self.num_lables)] for j in range(self.num_lables)]
        for i in range(len(_y)):
            confusion_matrix[_y[i]][predicts[i]] += 1

        df = pd.DataFrame(confusion_matrix, columns=['Pred' + str(i) for i in range(10)], 
                          index=['Class ' + str(i) for i in range(10)])
        print(df)

    def show_result(self, test_X, test_y):
        self.print_accuracy(self.train_X, self.train_y, test_X, test_y)
        print()
        print("Confusion Matrix of train data:")
        self.print_confusion_matrix(self.train_X, self.train_y)
        print()
        print("Confusion Matrix of test data:")
        self.print_confusion_matrix(test_X, test_y)
        print("\n_______________________\n\n")
    

In [6]:
def four_fold_CV(X, Y, fold_num=4):
    data_size = len(X)
    fold_size = int(data_size / fold_num)
    
    validation_sets = [None for i in range(fold_num)]
    train_sets = [None for i in range(fold_num)] 
    data = range(data_size)
    
    for i in range(fold_num):
        n_sample = fold_size
        if i == fold_num - 1:
            n_sample = data_size - (fold_num - 1) * fold_size
        fold_ind = np.array(np.random.choice(data, n_sample, replace=False))
        data = np.setdiff1d(data, fold_ind)
        
        validation_sets[i] = {'X': X[fold_ind], 'Y': Y[fold_ind]}
        rest_ind = np.setdiff1d(range(data_size), fold_ind)
        train_sets[i] = {'X': X[rest_ind], 'Y': Y[rest_ind]}
    
    return train_sets, validation_sets
    

In [19]:
print("Section B")
print("______________________________________")
print()

# CC = [0.1] + [i for i in range(1, 10)] + [10 * i for i in range(1, 11)]
CC = [0.1, 1.0, 10, 100.0]

accs = [0 for i in range(len(CC))]
folds_num = 4

kernel = LinearKernel()
train_sets, validation_sets = four_fold_CV(train_X, train_Y, folds_num)

for (cntC, C) in enumerate(CC):
    svc = MultiClass_SVC(C=C, kernel=kernel)
    
    avg_acc = 0
    for i in range(folds_num):
        t_X = train_sets[i]['X']
        t_Y = train_sets[i]['Y']
        v_X = train_sets[i]['X']
        v_Y = train_sets[i]['Y']
        
        try:
            svc.fit(t_X, t_Y)
            avg_acc += svc.get_accuracy(v_X, v_Y)
        except Exception as e: 
            print(e)
        
    accs[cntC] = avg_acc / folds_num
  

print(accs)
print("--------------")
print("-> Result of 4-fold cross validation for Linear Kernel:")
c_ind = np.argmax(accs)
best_C = CC[c_ind]
print("    Best parameter C is:", best_C, "\n", 
      "    with average validation accuracy =", accs[c_ind])
print()

print("-> Result of traing Soft Margin SVM using QP optimization with best learned parameter C =", str(best_C) + ":")
print()
svc = MultiClass_SVC(C=best_C, kernel=kernel)
svc.fit(train_X, train_Y)
svc.show_result(test_X, test_Y) 


Section B
______________________________________

[0.9153253978610382, 0.9590037978262406, 0.9796053740464655, 0.9903168344017268]
--------------
-> Result of 4-fold cross validation for Linear Kernel:
    Best parameter C is: 100.0 
     with average validation accuracy = 0.9903168344017268

-> Result of traing Soft Margin SVM using QP optimization with best learned parameter C = 100.0:

train accuracy score: 0.9894932014833128
test accuracy score: 0.9441340782122905

Confusion Matrix of train data:
         Pred0  Pred1  Pred2  Pred3  Pred4  Pred5  Pred6  Pred7  Pred8  Pred9
Class 0    161      0      0      0      0      0      0      0      0      0
Class 1      0    161      0      0      0      1      1      0      1      0
Class 2      0      0    160      0      0      0      0      0      0      0
Class 3      0      0      0    162      0      0      0      0      2      0
Class 4      0      0      0      0    164      0      0      0      0      0
Class 5      0      0     

In [13]:
print("Section P")
print("______________________________________")
print()

# ss = [0.1 * i for i in range(10, 0, -1)]
# CC = [10 * i for i in range(1, 11)]

ss = [0.1 * i for i in range(10, 0, -2)]
CC = [10., 100.]


accs = [[0 for j in range(len(ss))] for i in range(len(CC))]
folds_num = 4

train_sets, validation_sets = four_fold_CV(train_X, train_Y, folds_num)

for (cnts, s) in enumerate(ss):
    for (cntC, C) in enumerate(CC):
        kernel = GaussianKernel(theta=s)
        svc = MultiClass_SVC(C=C, kernel=kernel)

        avg_acc = 0
        for i in range(folds_num):
            t_X = train_sets[i]['X']
            t_Y = train_sets[i]['Y']
            v_X = train_sets[i]['X']
            v_Y = train_sets[i]['Y']

            try:
                svc.fit(t_X, t_Y)
                avg_acc += svc.get_accuracy(v_X, v_Y)
            except Exception as e: 
                print(e)

        accs[cntC][cnts] = avg_acc / folds_num

print(accs)
print("--------------")
print("-> Result of 4-fold cross validation for Gaussian Kernel:")

s_ind = int(np.argmax(accs) % len(ss))
c_ind = int((np.argmax(accs) - s_ind) / len(ss))
best_C = CC[c_ind]
best_s = ss[s_ind]

print("    Best parameter C is:", best_C, "\n",
      "   Best parameter sigm s:", best_s, "\n",
      "   with average validation accuracy =", accs[c_ind][s_ind])
print()

print("-> Result of traing Soft Margin SVM using QP optimization \n",
      "   with best learned parameter C =", best_C, "and sigma =", str(best_s) + ":")
print()

kernel = GaussianKernel(theta=best_s)
svc = MultiClass_SVC(C=best_C, kernel=kernel)
svc.fit(train_X, train_Y)
svc.show_result(test_X, test_Y) 

Section P
______________________________________

[[0.3515140331990365, 0.7011291532777659, 0.8920521582636023, 0.9719832156197498, 0.9995877985656886], [0.3515140331990365, 0.7011291532777659, 0.8920521582636023, 0.9719832156197498, 0.9995877985656886]]
--------------
-> Result of 4-fold cross validation for Gaussian Kernel:
    Best parameter C is: 10.0 
    Best parameter sigm s: 0.2 
    with average validation accuracy = 0.9995877985656886

-> Result of traing Soft Margin SVM using QP optimization 
    with best learned parameter C = 10.0 and sigma = 0.2:

train accuracy score: 0.9993819530284301
test accuracy score: 0.9888268156424581

Confusion Matrix of train data:
         Pred0  Pred1  Pred2  Pred3  Pred4  Pred5  Pred6  Pred7  Pred8  Pred9
Class 0    161      0      0      0      0      0      0      0      0      0
Class 1      0    163      0      0      0      0      0      0      1      0
Class 2      0      0    160      0      0      0      0      0      0      0
Class 